In [18]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import plotly.graph_objects as go
from plotting import CandlePlot
import pyarrow as pa
import pyarrow.parquet as pq
pd.set_option("display.max_columns", None)

In [19]:
class Data:
    
    def __init__(self, source):
        assert type(source) == str or type(source) == pd.DataFrame, 'Invalid source'
        if type(source) == str:
            self.df = {
                'raw': pd.read_pickle(source)
            }
        elif type(source) == pd.DataFrame:
            self.df = {
                'raw': source.copy()
            }            

        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]
        self.datalen = self.df['raw'].shape[0]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(pd.concat([df.head(2), df.tail(1)])) + '\n'
        return repr

    def prep_data(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows as preparation for fast data creation
        direction: 1 if data should be selected from top and -1 if from bottom
        '''
        assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'
        
        if cols == None:
            cols = self.df[source].columns
        if direction == 1:
            self.df[name] = self.df[source][cols].iloc[:rows].copy()
        else:
            self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: list):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        cols = exist_cols + cols
        self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None, add_cols: list=None):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fastdf = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata()
        '''
        
        self.prep_data(name=name, rows=rows, direction=direction, source=source, cols=cols)
        self.add_columns(name=name, cols=add_cols)

        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]
        self.fdatalen = len(self.fastdf[0])

    def fdata(self, column: str=None, index: int=None, rows: int=None):
        if column is None:
            return self.fastdf
        if index is None:
            return self.fastdf[self.fcols[column]]
        else:
            if rows:
                try:
                    return self.fastdf[self.fcols[column]][index:index+rows]
                except:
                    return self.fastdf[self.fcols[column]][index:]
            else:
                return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, index: int=None, value: float=None):
        assert value is not None, 'Value cannot be null'
        if index is None:
            assert len(value) == self.fdatalen
            for i in range(self.fdatalen):
                self.fastdf[self.fcols[column]][i] = value[i]
                print(i, )
        else:
            self.fastdf[self.fcols[column]][index] = value

In [20]:
PARQUET = 'D:/Trading/ml4t-data/EUR_USD-5M-price-movement-20-movements.parquet'
data = pq.read_table(PARQUET).to_pandas()

In [21]:
data.head(3)

,time,mid_c,pip_returns,entry_1,pips_1,entry_2,pips_2,entry_3,pips_3,entry_4,pips_4,entry_5,pips_5,entry_6,pips_6,entry_7,pips_7,entry_8,pips_8,entry_9,pips_9,entry_10,pips_10,entry_11,pips_11,entry_12,pips_12,entry_13,pips_13,entry_14,pips_14,entry_15,pips_15,entry_16,pips_16,entry_17,pips_17,entry_18,pips_18,entry_19,pips_19,entry_20,pips_20
0,2016-01-07 00:00:00,1.07786,NaN,0.0,23.9,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1
1,2016-01-07 00:05:00,1.07810,2.4,1.0,21.5,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1
2,2016-01-07 00:10:00,1.07828,1.8,2.0,25.1,20.0,20.5,70.0,-29.1,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1


In [22]:
MOVE_COUNT = 20

In [23]:
df = data.copy()

In [24]:
for i in range(MOVE_COUNT):
    df[f'trend_{i+1}'] = np.where(df[f'pips_{i+1}'] > 20, 1, 0)

In [25]:
def add_trends_all(data, r: range):
    data = data.copy()
    # trend_first = data[f'trend_{r[1]}']
    data[f'trend_{r[0]}_{r[0]}'] = data[f'trend_{r[0]}']
    for i in r:
        print(f'trend_{r[0]}_{i+1}')
        data[f'trend_{r[0]}_{i+1}'] = data[f'trend_{r[0]}_{i}'] + data[f'trend_{i+1}']

        # for j in range(i,i+1):
        #     print(j+1)
    return data

In [26]:
def extend_trends(data, start, end):
    trend = data[f'trend_{start}'].astype(str)
    for i in range(start, end):
        trend = trend + data[f'trend_{i+1}'].astype(str)
    return trend

In [27]:
df['trend_1_5'] = extend_trends(df, 1, 5)
df['trend_1_10'] = extend_trends(df, 1, 10)
df['trend_1_15'] = extend_trends(df, 1, 15)
df['trend_1_20'] = extend_trends(df, 1, 20)

In [28]:
df.head(3)

,time,mid_c,pip_returns,entry_1,pips_1,entry_2,pips_2,entry_3,pips_3,entry_4,pips_4,entry_5,pips_5,entry_6,pips_6,entry_7,pips_7,entry_8,pips_8,entry_9,pips_9,entry_10,pips_10,entry_11,pips_11,entry_12,pips_12,entry_13,pips_13,entry_14,pips_14,entry_15,pips_15,entry_16,pips_16,entry_17,pips_17,entry_18,pips_18,entry_19,pips_19,entry_20,pips_20,trend_1,trend_2,trend_3,trend_4,trend_5,trend_6,trend_7,trend_8,trend_9,trend_10,trend_11,trend_12,trend_13,trend_14,trend_15,trend_16,trend_17,trend_18,trend_19,trend_20,trend_1_5,trend_1_10,trend_1_15,trend_1_20
0,2016-01-07 00:00:00,1.07786,NaN,0.0,23.9,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100
1,2016-01-07 00:05:00,1.07810,2.4,1.0,21.5,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100
2,2016-01-07 00:10:00,1.07828,1.8,2.0,25.1,20.0,20.5,70.0,-29.1,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100


In [29]:
def add_proportions(df: pd.DataFrame):
    return pd.concat([df, (df / sum(df) * 100).round(decimals=4)], axis=1).set_axis(['count', 'proportion'], axis='columns')

def add_statistics(df: pd.DataFrame):
    return pd.concat([
        df.count(),
        df.count() / sum(df.count()) * 100,
        df.mean(),
        df.median(),
        df.max(),
        df.min()
    ], axis=1).set_axis(['count', 'proportion', 'mean', 'median', 'max', 'min'], axis='columns')

In [43]:
# add_proportions(df.groupby(['trend_1_10'])['time'].count()).to_csv('D:/Trading/ml4t-data/EUR_USD-5M-price-movement-20-movements.trends_1_10.csv')

In [94]:
UP = 1
DOWN = -1
def process_grid(trend, pips=20):
    price = 0
    long_entries = [price]
    short_entries = [price]
    closed_profits = list()
    for move in trend:
        move = UP if move == '1' else DOWN
        price = price + pips if move == UP else price - pips
        # print('price', price)
        if move == UP:
            # print('up')
            for index in range(len(long_entries)):
                # print(index)
                if long_entries[index] < price:
                    entry = long_entries.pop(index)
                    # print('entry', entry)
                    closed_profits.extend(price - entry)
        else: # move == DOWN
            # print('down')
            for index in range(len(short_entries)):
                # print(index)
                if short_entries[index] > price:
                    entry = short_entries.pop(index)
                    # print('entry', entry)
                    closed_profits.append(entry - price)
        long_entries.append(price)
        short_entries.append(price)
    
    long_losses = [price - entry for entry in long_entries]
    short_losses = [entry - price for entry in short_entries]
    return long_entries, long_losses, short_entries, short_losses, closed_profits

In [195]:
def print_positions(price, o_longs, o_shorts, c_longs, c_shorts):
    print(f'price, {price}, open_longs, {o_longs}, open_shorts, {o_shorts}, closed_longs, {c_longs}, closed_shorts, {c_shorts}')
    print(f'open positions, {sum(len(trades) for k, trades in o_longs.items()) + sum(len(trades) for k, trades in o_shorts.items())}')
    print(f'closed positions, {sum(len(trades) for k, trades in c_longs.items()) + sum(len(trades) for k, trades in c_shorts.items())}')

In [13]:
def find_positions(price, o_longs, o_shorts, c_longs, c_shorts):
    return dict(
        open=sum(len(trades) for k, trades in o_longs.items()) + sum(len(trades) for k, trades in o_shorts.items()),
        closed=sum(len(trades) for k, trades in c_longs.items()) + sum(len(trades) for k, trades in c_shorts.items())
    )

In [3]:
def calc_pnl(price, o_longs, o_shorts, c_longs, c_shorts, pips=20):
    closed_profit, open_loss = 0, 0
    for p, trades in c_longs.items():
        closed_profit = closed_profit + pips * len(trades)
    for p, trades in c_shorts.items():
        closed_profit = closed_profit + pips * len(trades)
    for p, trades in o_longs.items():
        open_loss = open_loss + (p - price) * len(trades)
    for p, trades in o_shorts.items():
        open_loss = open_loss + (price - p) * len(trades)
    return closed_profit, -open_loss

In [16]:
UP = 1
DOWN = -1
def process_grid(trend, pips=20):
    price = 0
    counter = 1
    o_longs = dict()
    o_shorts = dict()
    c_longs = dict()
    c_shorts = dict()
    o_longs[price] = [counter]
    o_shorts[price] = [counter]

    for move in trend:
        move = UP if move == '1' else DOWN
        price = price + pips if move == UP else price - pips
        # print('price', price)
        if move == UP:
            # print('up')
            in_profit_longs = list()
            for p, cntr in o_longs.items():
                if p < price:
                    # print(p, cntr)
                    if p in c_longs.keys():
                        c_longs[p].extend(cntr)
                    else:
                        c_longs[p] = cntr
                    in_profit_longs.append(p)
            for p in in_profit_longs:
                del o_longs[p]
            # print_positions(price, o_longs, o_shorts, c_longs, c_shorts)
        else: # move == DOWN
            # print('down')
            in_profit_shorts = list()
            for p, cntr in o_shorts.items():
                if p > price:
                    # print(p, cntr)
                    if p in c_shorts.keys():
                        c_shorts[p].extend(cntr)
                    else:
                        c_shorts[p] = cntr
                    in_profit_shorts.append(p)
            for p in in_profit_shorts:
                del o_shorts[p]
            # print_positions(price, o_longs, o_shorts, c_longs, c_shorts)
        counter += 1
        if price in o_longs.keys():
            o_longs[price].append(counter)
        else:
            o_longs[price] = [counter]
        if price in o_shorts.keys():
            o_shorts[price].append(counter)
        else:
            o_shorts[price] = [counter]
    # print_positions(price, o_longs, o_shorts, c_longs, c_shorts)

    closed_profit, open_loss = calc_pnl(price, o_longs, o_shorts, c_longs, c_shorts, pips)
    positions = find_positions(price, o_longs, o_shorts, c_longs, c_shorts)

    return dict(
        open_longs=o_longs,
        open_shorts=o_shorts,
        closed_longs=c_longs,
        closed_shorts=c_shorts,
        open_loss=open_loss,
        closed_profit=closed_profit,
        open_trades=positions['open'],
        closed_trades=positions['closed']
    )

In [189]:
# def pnl(trend, pips=20):
#     price, o_longs, o_shorts, c_longs, c_shorts = process_grid(trend, pips)
#     return calc_pnl(price, o_longs, o_shorts, c_longs, c_shorts, pips)

In [17]:
process_grid('100111100111100100100110', pips=20)

{'open_longs': {100: [14], 80: [15, 17], 60: [18, 20, 24], 40: [25]},
 'open_shorts': {-20: [4], 0: [5], 20: [6, 10, 22], 40: [23, 25]},
 'closed_longs': {0: [1, 3, 5],
  -20: [4],
  20: [2, 6, 10, 22],
  40: [7, 9, 11, 19, 21, 23],
  60: [8, 12, 16],
  80: [13]},
 'closed_shorts': {20: [2],
  0: [1, 3],
  60: [8, 12, 16, 18, 20, 24],
  40: [7, 9, 11, 19, 21],
  100: [14],
  80: [13, 15, 17]},
 'open_loss': -360,
 'closed_profit': 720,
 'open_trades': 14,
 'closed_trades': 36}

In [32]:
def open_loss(grid_results):
    return grid_results['open_loss']
def closed_profit(grid_results):
    return grid_results['closed_profit']
def open_trades(grid_results):
    return grid_results['open_trades']
def closed_trades(grid_results):
    return grid_results['closed_trades']

In [30]:
# df['grid_1_10'] = df['trend_1_10'].apply(process_grid, pips=20)
# df['grid_1_15'] = df['trend_1_15'].apply(process_grid, pips=20)
# df['grid_1_20'] = df['trend_1_20'].apply(process_grid, pips=20)

In [33]:
for i in [5, 10, 15, 20]:
    df[f'grid_1_{i}'] = df[f'trend_1_{i}'].apply(process_grid, pips=20)
    df[f'open_loss_1_{i}'] = df[f'grid_1_{i}'].apply(open_loss)
    df[f'closed_profit_1_{i}'] = df[f'grid_1_{i}'].apply(closed_profit)
    df[f'open_trades_1_{i}'] = df[f'grid_1_{i}'].apply(open_trades)
    df[f'closed_trades_1_{i}'] = df[f'grid_1_{i}'].apply(closed_trades)

In [34]:
df.head(3)

,time,mid_c,pip_returns,entry_1,pips_1,entry_2,pips_2,entry_3,pips_3,entry_4,pips_4,entry_5,pips_5,entry_6,pips_6,entry_7,pips_7,entry_8,pips_8,entry_9,pips_9,entry_10,pips_10,entry_11,pips_11,entry_12,pips_12,entry_13,pips_13,entry_14,pips_14,entry_15,pips_15,entry_16,pips_16,entry_17,pips_17,entry_18,pips_18,entry_19,pips_19,entry_20,pips_20,trend_1,trend_2,trend_3,trend_4,trend_5,trend_6,trend_7,trend_8,trend_9,trend_10,trend_11,trend_12,trend_13,trend_14,trend_15,trend_16,trend_17,trend_18,trend_19,trend_20,trend_1_5,trend_1_10,trend_1_15,trend_1_20,grid_1_5,open_loss_1_5,closed_profit_1_5,open_trades_1_5,closed_trades_1_5,grid_1_10,open_loss_1_10,closed_profit_1_10,open_trades_1_10,closed_trades_1_10,grid_1_15,open_loss_1_15,closed_profit_1_15,open_trades_1_15,closed_trades_1_15,grid_1_20,open_loss_1_20,closed_profit_1_20,open_trades_1_20,closed_trades_1_20
0,2016-01-07 00:00:00,1.07786,NaN,0.0,23.9,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100,"{'open_longs': {40: [3], 20: [4, 6]}, 'open_sh...",-60,120,6,6,"{'open_longs': {60: [8, 10], 40: [11]}, 'open_...",-140,260,9,13,"{'open_longs': {120: [15], 100: [16]}, 'open_s...",-540,400,12,20,"{'open_longs': {120: [15], 100: [16], 80: [17,...",-340,580,13,29
1,2016-01-07 00:05:00,1.07810,2.4,1.0,21.5,19.0,21.1,39.0,-24.3,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100,"{'open_longs': {40: [3], 20: [4, 6]}, 'open_sh...",-60,120,6,6,"{'open_longs': {60: [8, 10], 40: [11]}, 'open_...",-140,260,9,13,"{'open_longs': {120: [15], 100: [16]}, 'open_s...",-540,400,12,20,"{'open_longs': {120: [15], 100: [16], 80: [17,...",-340,580,13,29
2,2016-01-07 00:10:00,1.07828,1.8,2.0,25.1,20.0,20.5,70.0,-29.1,88.0,-20.7,97.0,29.0,107.0,23.6,109.0,34.6,113.0,-25.0,120.0,20.8,137.0,-33.9,191.0,21.5,196.0,20.3,207.0,30.2,223.0,31.3,239.0,-20.0,302.0,-26.2,304.0,-23.3,368.0,25.1,383.0,-21.3,410.0,-49.1,1,1,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,0,11001,1100111010,110011101011110,11001110101111000100,"{'open_longs': {40: [3], 20: [4, 6]}, 'open_sh...",-60,120,6,6,"{'open_longs': {60: [8, 10], 40: [11]}, 'open_...",-140,260,9,13,"{'open_longs': {120: [15], 100: [16]}, 'open_s...",-540,400,12,20,"{'open_longs': {120: [15], 100: [16], 80: [17,...",-340,580,13,29


In [47]:
def display_grid(row, trend):
    print(f'trend_{trend}:', row[f'trend_{trend}'])
    print(f'grid_{trend}:', row[f'grid_{trend}'])
    print(f'open_loss_{trend}:', row[f'open_loss_{trend}'])
    print(f'open_trades_{trend}:', row[f'open_trades_{trend}'])
    print(f'closed_profit_{trend}:', row[f'closed_profit_{trend}'])
    print(f'closed_trades_{trend}:', row[f'closed_trades_{trend}'])

# df.iloc[0]['trend_1_5'], df.iloc[0]['grid_1_5'], df.iloc[0][['open_loss_1_5', 'closed_profit_1_5', 'open_trades_1_5', 'closed_trades_1_5']]

In [50]:
display_grid(df.iloc[0], '1_20')

trend_1_20: 11001110101111000100
grid_1_20: {'open_longs': {120: [15], 100: [16], 80: [17, 19], 60: [20], 40: [21]}, 'open_shorts': {0: [1, 5], 20: [6], 40: [7, 9, 11, 21]}, 'closed_longs': {0: [1, 5], 20: [2, 4, 6], 40: [3, 7, 9, 11], 60: [8, 10, 12, 18], 80: [13], 100: [14]}, 'closed_shorts': {40: [3], 20: [2, 4], 60: [8, 10, 12, 18, 20], 120: [15], 100: [14, 16], 80: [13, 17, 19]}, 'open_loss': -340, 'closed_profit': 580, 'open_trades': 13, 'closed_trades': 29}
open_loss_1_20: -340
open_trades_1_20: 13
closed_profit_1_20: 580
closed_trades_1_20: 29


In [52]:
add_proportions(df.groupby(['trend_1_5'])['time'].count())

,count,proportion
trend_1_5,,
00000,23266,3.9148
00001,24508,4.1238
00010,22546,3.7937
00011,17967,3.0232
00100,21310,3.5857
00101,19141,3.2207
00110,16998,2.8602
00111,15646,2.6327
01000,23235,3.9096


In [77]:
def grid_statistics(df: pd.DataFrame, trend):
    columns = ['count', 'proportion', 'open_loss_mean', 'open_loss_min', \
        'open_trades_mean', 'open_trades_max', 'closed_profit_mean', 'closed_profit_max', 'closed_profit_min', \
        'closed_trades_mean', 'closed_trades_max']
    return pd.concat([
        df['time'].count(),
        df['time'].count() / sum(df['time'].count()) * 100,
        df[f'open_loss_{trend}'].mean(),
        df[f'open_loss_{trend}'].min(),
        df[f'open_trades_{trend}'].mean(),
        df[f'open_trades_{trend}'].max(),
        df[f'closed_profit_{trend}'].mean(),
        df[f'closed_profit_{trend}'].max(),
        df[f'closed_profit_{trend}'].min(),
        df[f'closed_trades_{trend}'].mean(),
        df[f'closed_trades_{trend}'].max()
    ], axis=1).set_axis(columns, axis='columns')

In [78]:
grid_statistics(df.groupby(['trend_1_5']), '1_5')

,count,proportion,open_loss_mean,open_loss_min,open_trades_mean,open_trades_max,closed_profit_mean,closed_profit_max,closed_profit_min,closed_trades_mean,closed_trades_max
trend_1_5,,,,,,,,,,,
00000,23266,3.914838,-300.0,-300,7.0,7,100.0,100,100,5.0,5
00001,24508,4.123822,-140.0,-140,7.0,7,100.0,100,100,5.0,5
00010,22546,3.793688,-140.0,-140,7.0,7,100.0,100,100,5.0,5
00011,17967,3.023205,-80.0,-80,6.0,6,120.0,120,120,6.0,6
00100,21310,3.585713,-160.0,-160,6.0,6,120.0,120,120,6.0,6
00101,19141,3.220748,-60.0,-60,7.0,7,100.0,100,100,5.0,5
00110,16998,2.860157,-60.0,-60,6.0,6,120.0,120,120,6.0,6
00111,15646,2.632664,-120.0,-120,5.0,5,140.0,140,140,7.0,7
01000,23235,3.909622,-180.0,-180,6.0,6,120.0,120,120,6.0,6


In [79]:
EXCEL = 'D:/Trading/ml4t-data/EUR_USD-5M-price-movement-20-movements.grids.stats.xlsx'
writer = pd.ExcelWriter(EXCEL,  engine="xlsxwriter")

for trend in ['1_5', '1_10', '1_15', '1_20']:
    group = grid_statistics(df.groupby([f'trend_{trend}']), trend).to_excel(writer, sheet_name=trend)

writer.close()